In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from itertools import chain
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import numpy as np
import re
from numpy import random

In [20]:
all_countries=pd.read_csv("C:/Users/homa/Python-Notbook/TradeMap/all_countries.csv")
Product_Code=pd.read_csv("C:/Users/homa/Python-Notbook/TradeMap/Product_Codem.csv")
all_countries.head(2)

,Country,Unnamed: 1,Country1,Unnamed: 3
0,'Afghanistan,NaN,Afghanistan,NaN
1,'Albania,NaN,Albania,NaN


In [21]:
driver= webdriver.Chrome()
driver.get("https://www.trademap.org/Index.aspx")
#driver.maximize_window()
waits= WebDriverWait(driver, 30)

In [22]:
def login():
    try:
        
        logintext=driver.find_element(By.CSS_SELECTOR ,("#ctl00_MenuControl_Label_Login"))
        if logintext.text=='Login': 
                
            login=waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_MenuControl_Label_Login')))
            login.click()
            url= driver.current_url
            if url!='https://www.trademap.org/Index.aspx': 
                getemail= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#Username')))
                getemail.send_keys('homa.abedinzadeh@gmail.com')
                #time.sleep(random.randint(5,20))
                getpassword= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#Password')))
                getpassword.send_keys('Aa@97019701')
                #time.sleep(random.randint(5,20))
                waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.container > div > div:nth-child(2) > div > div > div > form > fieldset > div:nth-child(4) > div > button'))).click()
            
    
    except Exception as e:
                print(f"Error selecting option : {e}")


In [23]:
def delete():
    waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Image_deleteProduct'))).click()
    waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Image_DeleteCountry'))).click()  

In [24]:
df = pd.DataFrame()
temp = pd.DataFrame()
for i in range(len(Product_Code)):
    try:
        login()
        delete()
        inputproduct = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_Input')))
        inputproduct.send_keys(Product_Code['ProductCode'].iloc[i])
        time.sleep(random.randint(3,6))
        proclick = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_c0'))).click()
        time.sleep(random.randint(2,3))
        btn = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Button_TradeIndicators'))).click()
        logintext=driver.find_element(By.CSS_SELECTOR ,("#ctl00_MenuControl_Label_Login"))
        if logintext.text=='Login':  
            login()


        dropdown_element = waits.until(EC.visibility_of_element_located((By.ID, 'ctl00_NavigationControl_DropDownList_Country')))  # Replace with actual ID     
        select = Select(dropdown_element)
        options = select.options
        for j in range(len(options)):  
                print(j)
                try:
                    
                    dropdown_element = waits.until(EC.visibility_of_element_located((By.ID, 'ctl00_NavigationControl_DropDownList_Country')))
                    select = Select(dropdown_element)
                    options = select.options
                    option_text = options[j].text

                    if option_text != 'All' or option_text.find('elsewhere ')!=-1  :
                        select.select_by_visible_text(option_text)
                        # تغییر تعداد رکوردها به 300
                        incshow = waits.until(EC.element_to_be_clickable((By.XPATH, "//select[@id='ctl00_PageContent_GridViewPanelControl_DropDownList_PageSize']")))
                        drop = Select(incshow)
                        
                       
                        drop.select_by_value('300')





                    # استخراج جدول
                        all_tables = pd.read_html(driver.page_source, attrs={'id': 'ctl00_PageContent_MyGridView1'})
                        temp = all_tables[0]

                        if temp.shape[0] != 1:
                            temp = temp.iloc[1:, 1:].iloc[:, :-4]
                            new_header = temp.iloc[0]
                            temp = temp[1:]
                            temp.columns = new_header

                            # افزودن اطلاعات کشور و کد محصول
                            temp['Importer'] = option_text
                            temp['productcode'] = re.sub(r'[^\w]', ' ', Product_Code['ProductCode'].iloc[i])
                            temp.drop(temp[temp['Exporters'] == 'Total'].index, inplace=True)
                            temp.drop(temp[temp['Exporters'] == 'World'].index, inplace=True)

                            # تغییر نام ستون‌ها
                            average = temp.columns[15]
                            sharein = temp.columns[3]
                            temp.rename(columns={average: 'Average tariff%', sharein: 'ShareinCounryImports'}, inplace=True)

                            # افزودن به DataFrame اصلی
                            df = pd.concat([df, temp], axis=0)
                            temp = pd.DataFrame()

                            print(f"Selected option: {option_text}, {j}")
                        time.sleep(random.randint(1,3))
                    if (j+1)%10 ==0:
                            #driver.quit()
                            #driver= webdriver.Chrome()
                            waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_MenuControl_Label_Menu_Home'))).click()
                            time.sleep(random.randint(2,5))
                            login()
                            delete()
                            time.sleep(random.randint(3,5))
                            inputproduct = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_Input')))
                            inputproduct.send_keys(Product_Code['ProductCode'].iloc[i])
                            time.sleep(random.randint(3,6))
                            proclick = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_c0'))).click()
                            time.sleep(random.randint(2,3))
                            btn = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Button_TradeIndicators'))).click()
                            
                
                except Exception as e:
                    print(f"Error selecting option {j}: {e}")
                    site = driver.current_url
                    if site!= 'https://www.trademap.org/Index.aspx':
                        driver.get('https://www.trademap.org/Index.aspx')
                    login()
                    delete()
                    time.sleep(random.randint(3,5))
                    inputproduct = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_Input')))
                    inputproduct.send_keys(Product_Code['ProductCode'].iloc[i])
                    time.sleep(random.randint(3,6))
                    proclick = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_c0'))).click()
                    time.sleep(random.randint(2,3))
                    btn = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Button_TradeIndicators'))).click()
                    j-=1
    except:
         print('Get All Country ')
    waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_MenuControl_Label_Menu_Home'))).click()
    #delete()
    time.sleep(30)
    driver.refresh()
    i+=1


0
1
Selected option: Afghanistan, 1
2
3
Selected option: Albania, 3
4
Selected option: Algeria, 4
5
6
7
Selected option: Andorra, 7
8
Selected option: Angola, 8
9
Selected option: Anguilla, 9
10
Selected option: Antigua and Barbuda, 10
11
Selected option: Area Nes, 11
12
Selected option: Argentina, 12
13
Selected option: Armenia, 13
14
Selected option: Aruba, 14
15
16
Selected option: Australia, 16
17
Selected option: Austria, 17
18
Selected option: Azerbaijan, 18
19
Selected option: Bahamas, 19
20
Selected option: Bahrain, 20
21
Selected option: Bangladesh, 21
22
Selected option: Barbados, 22
23
Selected option: Belarus, 23
24
Selected option: Belgium, 24
25
Selected option: Belize, 25
26
Selected option: Benin, 26
27
Selected option: Bermuda, 27
28
Selected option: Bhutan, 28
29
Selected option: Bolivia, Plurinational State of, 29
30
Selected option: Bonaire, Sint Eustatius and Saba, 30
31
Selected option: Bosnia and Herzegovina, 31
32
Selected option: Botswana, 32
33
34
Selected opt

KeyboardInterrupt: 

In [25]:
df.shape

(141501, 18)

In [26]:
df.head()

1,Exporters,Value imported in 2023 (USD thousand),Trade balance 2023 (USD thousand),ShareinCounryImports,Quantity imported in 2023,Quantity unit,Unit value (USD/unit),"Growth in imported value between 2019-2023 (%, p.a.)","Growth in imported quantity between 2019-2023 (%, p.a.)","Growth in imported value between 2022-2023 (%, p.a.)",Ranking of partner countries in world exports,Share of partner countries in world exports (%),"Total exports growth in value of partner countries between 2019-2023 (%, p.a.)",Average distance between partner countries and all their importing markets (km),Concentration of all importing countries of partner countries,Average tariff%,Importer,productcode
3,Czech Republic,NaN,167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33,0.1,9,280,0.42,10,Afghanistan,200290
4,United States of America,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,6.1,8,5410,0.14,10,Afghanistan,200290
5,Andorra,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,Afghanistan,200290
6,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,26.7,18,7878,0.05,10,Afghanistan,200290
7,Italy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,25.1,13,2253,0.09,10,Afghanistan,200290


In [30]:
temp.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,Information not available,Information not available,Information not available,Information not available,Information not available,Information not available,Information not available,Information not available,Information not available,Information not available,...,Information not available,Information not available,Information not available,Information not available,Information not available,Information not available,Information not available,Information not available,Information not available,Information not available


In [36]:
df[df['Exporters'] =='World']

1,Exporters,Value imported in 2023 (USD thousand),Trade balance 2023 (USD thousand),ShareinCounryImports,Quantity imported in 2023,Quantity unit,Unit value (USD/unit),"Growth in imported value between 2019-2023 (%, p.a.)","Growth in imported quantity between 2019-2023 (%, p.a.)","Growth in imported value between 2022-2023 (%, p.a.)",Ranking of partner countries in world exports,Share of partner countries in world exports (%),"Total exports growth in value of partner countries between 2019-2023 (%, p.a.)",Average distance between partner countries and all their importing markets (km),Concentration of all importing countries of partner countries,Average tariff%,Importer,productcode
2,World,129,-129,100,79,Tons,1633,37,24,90,NaN,100,16,NaN,NaN,NaN,Andorra,200290
2,World,302,-302,100,74,Tons,4081,10,-4,14,NaN,100,16,NaN,NaN,NaN,Antigua and Barbuda,200290
2,World,16300,-15252,100,7899,Tons,2064,-18,-33,47,NaN,100,16,NaN,NaN,NaN,Argentina,200290
2,World,9709,-1903,100,8748,Tons,1110,30,19,394,NaN,100,16,NaN,NaN,NaN,Armenia,200290
2,World,233,-233,100,107,Tons,2178,5,-2,28,NaN,100,16,NaN,NaN,NaN,Aruba,200290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,World,41,788560,100,5,Carats,8200,-36,-66,825,NaN,100,-14,NaN,NaN,NaN,Canada,71021
2,World,NaN,82,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,-14,NaN,NaN,NaN,China,71021
2,World,4,-4,100,0,Tons,NaN,-24,NaN,-60,NaN,100,-14,NaN,NaN,NaN,Costa Rica,71021
2,World,24,-24,100,0,Tons,NaN,NaN,NaN,NaN,NaN,100,-14,NaN,NaN,NaN,Croatia,71021
